In [1]:
# Generate data for the Darcy flow problem
# Input: a - the permeability
# output: u - the flow through the permeability
# data generated:
# 1000 training pairs
# 6 sets of 200 testing pairs, with resolutions of:
# 256, 1024, 224, 192, 160, 128

In [ ]:
pip install torch

In [ ]:
pip install tensorflow

In [ ]:
pip install --user git+https://github.com/devitocodes/devito.git

In [7]:
import torch
import math
import devito

import numpy as np

from random_fields import GaussianRF

from timeit import default_timer

import scipy.io

2022-07-28 08:50:55.376686: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-28 08:50:55.376741: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [8]:
from matplotlib import pyplot, cm

def plot_field(field, xmin=0., xmax=1., ymin=0., ymax=1., zmin=None, zmax=None,
               view=None, linewidth=0):
    """
    Utility plotting routine for 2D data.
    Parameters
    ----------
    field : array_like
        Field data to plot.
    xmax : int, optional
        Length of the x-axis.
    ymax : int, optional
        Length of the y-axis.
    view: int, optional
        View point to intialise.
    """
    if xmin > xmax or ymin > ymax:
        raise ValueError("Dimension min cannot be larger than dimension max.")
    if (zmin is not None and zmax is not None):
        if zmin > zmax:
            raise ValueError("Dimension min cannot be larger than dimension max.")
    elif(zmin is None and zmax is not None):
        if np.min(field) >= zmax:
            warning("zmax is less than field's minima. Figure deceptive.")
    elif(zmin is not None and zmax is None):
        if np.max(field) <= zmin:
            warning("zmin is larger than field's maxima. Figure deceptive.")
    x_coord = np.linspace(xmin, xmax, field.shape[0])
    y_coord = np.linspace(ymin, ymax, field.shape[1])
    fig = pyplot.figure(figsize=(11, 7), dpi=100)
    ax = fig.gca(projection='3d')
    X, Y = np.meshgrid(x_coord, y_coord, indexing='ij')
    ax.plot_surface(X, Y, field[:], cmap=cm.viridis, rstride=1, cstride=1,
                    linewidth=linewidth, antialiased=False)

    # Enforce axis measures and set view if given
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    if zmin is None:
        zmin = np.min(field)
    if zmax is None:
        zmax = np.max(field)
    ax.set_zlim(zmin, zmax)

    if view is not None:
        ax.view_init(*view)

    # Label axis
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')

    pyplot.show()

In [12]:

'''
function to generate 'u' from 'a' using Devito
parameters 
-----------------
perm: tensor of size (1, s, s)
    this is "a"
f: tensor of size (s, s)
    the forcing function f(x) = 1
s: int
    the resolution
 '''

def darcy_flow_2d(perm, f, s):
    
    # a(x) is the coefficients
    # f is the forcing function
    # give the a and f1 functions the same data as inputs perm and f
    for i in range(s):
        f1.data[i] = f[i]
        a.data[i] = perm[0][i]
    
    # call operator for the 1000th psuedo-timestep
    op(time=1000)
    #plot_field(u.data[0])
    #plot_field(a.data)
    #print(u.data[0])
   
    
    return torch. from_numpy(u.data[0])

In [ ]:

# number of grid points on [0,1]^2 
# s = 256
s = 1024 
# s = 224
# s = 192
# s = 160
# s = 128

# Number of solutions to generate
# N = 1000
N = 200

# Set up 2d GRF with covariance parameters to generate random coefficients
norm_a = GaussianRF(2, s, alpha=2, tau=3)

# Forcing function, f(x) = 1 
f = np.ones((s, s))

# create s x s grid with spacing 1
grid = devito.Grid(shape=(s, s), extent=(1.0,1.0))
    
# create function on grid
# space order of 2 to enable 2nd derivative
# TimeFunction for u can be used despite the lack of a time-dependence. This is done for psuedotime
u = devito.TimeFunction(name='u', grid=grid, space_order=2)
a = devito.Function(name='a', grid=grid, space_order=2)
f1 = devito.Function(name='f1', grid=grid, space_order=2)

# define 2D Darcy flow equation
equation_u = devito.Eq(-(a*u.dxc).dxc, f1)
    
# Let SymPy solve for the central stencil point
stencil = devito.solve(equation_u, u)
    
# Let our stencil populate the buffer `u.forward`
update = devito.Eq(u.forward, stencil)
    
    
# Boundary Conds. u=0 for all sides
x, y = grid.dimensions
t = grid.stepping_dim
    
nx = s
ny = s
bc = [devito.Eq(u[t+1, x, 0], 0.)]
bc += [devito.Eq(u[t+1, x, ny-1], 0.)]
bc += [devito.Eq(u[t+1, 0, y], 0.)]
bc += [devito.Eq(u[t+1, nx-1, y], 0.)]
    
op = devito.Operator([update] + bc)


# all the inputs
coeff = torch.zeros(N, s, s)
#all the solutions
sol = torch.zeros(N, s, s)

c = 0
t0 = default_timer()

for j in range(N):

    #Sample random fields
    # Exponentiate it, so that a(x) > 0
    # This is done so that the PDE is elliptic
    lognorm_a = torch.exp(norm_a.sample(1))
    w0 = lognorm_a
    
    # or create a threshold, either 4 or 12 (more common for permeability)
    thresh_a = norm_a.sample(1)
    for i in range(s):
        for b in range(s):
            if thresh_a[0][i][b] < 0:
                thresh_a[0][i][b] = 4
            else:
                    thresh_a[0][i][b] = 12
    w1 = thresh_a

    #Solve df either for w0 (exp perm) or w1 (thresh perm)
    output = darcy_flow_2d(w1, f, s)
    
    
    coeff[c:(c+1),...] = w1
    sol[c:(c+1),...] = output

    c += 1
    t1 = default_timer()
    print("sample no. :", c, "Time taken:", t1-t0)

# scipy.io.savemat('df_train.mat', mdict={'coeff': coeff.cpu().numpy(), 'sol': sol.cpu().numpy()})
# scipy.io.savemat('df_test_256.mat', mdict={'coeff': coeff.cpu().numpy(), 'sol': sol.cpu().numpy()})
scipy.io.savemat('df_test_1024.mat', mdict={'coeff': coeff.cpu().numpy(), 'sol': sol.cpu().numpy()})
# scipy.io.savemat('df_test_224.mat', mdict={'coeff': coeff.cpu().numpy(), 'sol': sol.cpu().numpy()})
# scipy.io.savemat('df_test_192.mat', mdict={'coeff': coeff.cpu().numpy(), 'sol': sol.cpu().numpy()})
# scipy.io.savemat('df_test_160.mat', mdict={'coeff': coeff.cpu().numpy(), 'sol': sol.cpu().numpy()})
# scipy.io.savemat('df_test_128.mat', mdict={'coeff': coeff.cpu().numpy(), 'sol': sol.cpu().numpy()})

Equation is not affine w.r.t the target, falling back to standardsympy.solve that may be slow
Operator `Kernel` ran in 1.72 s


sample no. : 1 Time taken: 39.03131002606824


Operator `Kernel` ran in 1.69 s


sample no. : 2 Time taken: 77.4026481313631


Operator `Kernel` ran in 1.68 s


...

sample no. : 100 Time taken: 3841.122359392233


Operator `Kernel` ran in 1.70 s


sample no. : 101 Time taken: 3879.2810072870925


Operator `Kernel` ran in 1.70 s


sample no. : 102 Time taken: 3917.7919501373544
